<a href="https://colab.research.google.com/github/rodjfg/los-fabulosos-pixelotls/blob/master/glm_together.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Left / Right model

In [1]:
!pip install --quiet git+https://github.com/rodjfg/los-fabulosos-pixelotls

In [2]:
from los_fabulosos_pixelotls.tools import load_raw_data, select_by_areas, select_by_contrast, select_trials, calculate_mean_firing_rate, select_by_response
import numpy as np
from matplotlib import rcParams 
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
import itertools

rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [3]:
# get data
alldat = load_raw_data('../dat')

In [4]:
def get_samples(selected_regions = ["VISp"], gaussfilter = True, gauss_sigma = 1, y_window = np.arange(250), contrast_list=None):

  if not contrast_list:
    contrast_list = [x for x in itertools.permutations([0,0.25,0.5,1], 2)]
  elif contrast_list == 'right_stimulus':
    contrast_list = [x for x in itertools.permutations([0,0.25,0.5,1], 2) if x[0] < x[1]]
  elif contrast_list == 'left_stimulus':
    contrast_list = [x for x in itertools.permutations([0,0.25,0.5,1], 2) if x[0] > x[1]]

  samples = np.empty((0,len(y_window)))
  y = np.empty((0,))

  for dat in alldat:
    dt = dat['bin_size']
    neurons_indices_in_area = select_by_areas(dat, selected_regions = selected_regions)

    if len(neurons_indices_in_area) > 0:
      for contrast in contrast_list:
        right_response_trials = select_trials(dat, contrast_pair=contrast, response_type = 'to_left') # trials in which mouse should move to the left (higher contrast on right side)
        left_response_trials = select_trials(dat, contrast_pair=contrast, response_type = 'to_right') # trials in which mouse should move to the right (higher contrast on left side)

        # average firing rate across neuron population (e.g. VISp) for each trial in which mouse moved to the left
        right_firing_rates = calculate_mean_firing_rate(dat['spks'][neurons_indices_in_area][:,right_response_trials][...,y_window], dt, ['population'], gaussfilter = gaussfilter, gauss_sigma = gauss_sigma)
        # average firing rate across neuron population (e.g. VISp) for each trial in which mouse moved to the right
        left_firing_rates = calculate_mean_firing_rate(dat['spks'][neurons_indices_in_area][:,left_response_trials][...,y_window], dt, ['population'], gaussfilter = gaussfilter, gauss_sigma = gauss_sigma)

        # find minimum number of samples that can be extracted from both conditions
        n_min = min(right_firing_rates.shape[0],left_firing_rates.shape[0])
        samples = np.concatenate((samples,right_firing_rates[:n_min,...]),axis=0)
        samples = np.concatenate((samples,left_firing_rates[:n_min,...]),axis=0)

        # 1: mouse moved to the left, 0: mouse moved to the right
        y = np.concatenate( (y, np.ones(n_min)) )
        y = np.concatenate( (y, np.zeros(n_min)) )

  return samples, y

## Quick test with LogisticRegressionCV

In [5]:
samples, y = get_samples(["VISp"])
samples_train, samples_test, y_train, y_test = train_test_split(samples, y, test_size=0.23, random_state=42)
clf = LogisticRegressionCV(penalty='l2', Cs=5, solver='sag', tol=0.0001, max_iter=5000, random_state=42).fit(samples_train, y_train)
print(f'C {C}, train {clf.score(samples_train,y_train):.2f}, test {clf.score(samples_test,y_test):.2f}')
clf = LogisticRegressionCV(penalty='l1', Cs=5, solver='saga', tol=0.0001, max_iter=5000, random_state=42).fit(samples_train, y_train)
print(f'C {clf.C_[0]}, train {clf.score(samples_train,y_train):.2f}, test {clf.score(samples_test,y_test):.2f}')

0.1: train 0.89, test 0.48
0.5: train 0.93, test 0.46
1.0: train 0.97, test 0.51
1.5: train 0.98, test 0.49


## Visual cortex areas loop

In [ ]:
regions_of_interest = [["VISa"], ["VISam"], ["VISl"], ["VISp"], ["VISpm"], ["VISrl"]]

In [9]:
model_list = []
for region in regions_of_interest:
    samples, y = get_samples(region)
    samples_train, samples_test, y_train, y_test = train_test_split(samples, y, test_size=0.23, random_state=42)
    clf = LogisticRegressionCV(penalty='l2', Cs=10, cv=3, solver='sag', tol=0.0001, max_iter=5000, random_state=42).fit(samples_train, y_train)
    print(f'{region}: C {clf.C_[0]}, train {clf.score(samples_train,y_train):.2f}, test {clf.score(samples_test,y_test):.2f}')
    model_list.append(clf)

['VISa']: C 0.3593813663804626, train 1.00, test 0.61
['VISam']: C 0.000774263682681127, train 0.75, test 0.41
['VISl']: C 0.0001, train 0.57, test 0.39
['VISp']: C 0.046415888336127774, train 0.85, test 0.46
['VISpm']: C 0.046415888336127774, train 0.97, test 0.53
['VISrl']: C 0.0001, train 0.52, test 0.44


## Smaller window around second peak

In [10]:
model_list = []
for region in regions_of_interest:
    samples, y = get_samples(region,y_window=np.arange(60,90))
    samples_train, samples_test, y_train, y_test = train_test_split(samples, y, test_size=0.23, random_state=42)
    clf = LogisticRegressionCV(penalty='l2', Cs=10, cv=3, solver='sag', tol=0.0001, max_iter=5000, random_state=42).fit(samples_train, y_train)
    print(f'{region}: C {clf.C_[0]}, train {clf.score(samples_train,y_train):.2f}, test {clf.score(samples_test,y_test):.2f}')
    model_list.append(clf)

['VISa']: C 0.0001, train 0.51, test 0.48
['VISam']: C 2.782559402207126, train 0.69, test 0.57
['VISl']: C 0.0001, train 0.52, test 0.43
['VISp']: C 0.005994842503189409, train 0.60, test 0.54
['VISpm']: C 21.54434690031882, train 0.89, test 0.68
['VISrl']: C 10000.0, train 0.83, test 0.33


## Dumb model with window before stimulus

In [11]:
model_list = []
for region in regions_of_interest:
    samples, y = get_samples(region,y_window=np.arange(0,50))
    samples_train, samples_test, y_train, y_test = train_test_split(samples, y, test_size=0.23, random_state=42)
    clf = LogisticRegressionCV(penalty='l2', Cs=10, cv=3, solver='sag', tol=0.0001, max_iter=5000, random_state=42).fit(samples_train, y_train)
    print(f'{region}: C {clf.C_[0]}, train {clf.score(samples_train,y_train):.2f}, test {clf.score(samples_test,y_test):.2f}')
    model_list.append(clf)

['VISa']: C 2.782559402207126, train 0.84, test 0.48
['VISam']: C 166.81005372000558, train 0.76, test 0.39
['VISl']: C 166.81005372000558, train 0.80, test 0.32
['VISp']: C 0.005994842503189409, train 0.63, test 0.49
['VISpm']: C 0.000774263682681127, train 0.52, test 0.63
['VISrl']: C 0.046415888336127774, train 0.75, test 0.44


## Full window, extremely smoothed

In [12]:
model_list = []
for region in regions_of_interest:
    samples, y = get_samples(region,gauss_sigma=5)
    samples_train, samples_test, y_train, y_test = train_test_split(samples, y, test_size=0.23, random_state=42)
    clf = LogisticRegressionCV(penalty='l2', Cs=10, cv=3, solver='sag', tol=0.0001, max_iter=5000, random_state=42).fit(samples_train, y_train)
    print(f'{region}: C {clf.C_[0]}, train {clf.score(samples_train,y_train):.2f}, test {clf.score(samples_test,y_test):.2f}')
    model_list.append(clf)

['VISa']: C 0.046415888336127774, train 0.73, test 0.43
['VISam']: C 0.0001, train 0.56, test 0.35
['VISl']: C 0.0001, train 0.52, test 0.43
['VISp']: C 0.000774263682681127, train 0.61, test 0.52
['VISpm']: C 0.046415888336127774, train 0.72, test 0.53
['VISrl']: C 0.0001, train 0.52, test 0.44


## Full window, no smoothing

In [13]:
model_list = []
for region in [["VISa"], ["VISam"], ["VISl"], ["VISp"], ["VISpm"], ["VISrl"]]:
    samples, y = get_samples(region,gaussfilter=False)
    samples_train, samples_test, y_train, y_test = train_test_split(samples, y, test_size=0.23, random_state=42)
    clf = LogisticRegressionCV(penalty='l2', Cs=10, cv=3, solver='sag', tol=0.0001, max_iter=5000, random_state=42).fit(samples_train, y_train)
    print(f'{region}: C {clf.C_[0]}, train {clf.score(samples_train,y_train):.2f}, test {clf.score(samples_test,y_test):.2f}')
    model_list.append(clf)

['VISa']: C 0.0001, train 0.75, test 0.57
['VISam']: C 0.0001, train 0.73, test 0.35
['VISl']: C 0.0001, train 0.60, test 0.43
['VISp']: C 0.0001, train 0.73, test 0.52
['VISpm']: C 0.046415888336127774, train 1.00, test 0.68
['VISrl']: C 0.0001, train 0.52, test 0.44


## Only for stimuli stronger on one side

## ... Right stimulus stronger than Left

In [14]:
model_list = []
for region in [["VISa"], ["VISam"], ["VISl"], ["VISp"], ["VISpm"], ["VISrl"]]:
    samples, y = get_samples(region, contrast_list='right_stimulus')
    samples_train, samples_test, y_train, y_test = train_test_split(samples, y, test_size=0.2, random_state=42)
    clf = LogisticRegressionCV(penalty='l2', Cs=10, cv=3, solver='sag', tol=0.0001, max_iter=5000, random_state=42).fit(samples_train, y_train)
    print(f'{region}: C {clf.C_[0]}, train {clf.score(samples_train,y_train):.2f}, test {clf.score(samples_test,y_test):.2f}')
    model_list.append(clf)

['VISa']: C 0.005994842503189409, train 0.93, test 0.12
['VISam']: C 0.000774263682681127, train 0.75, test 0.65
['VISl']: C 0.0001, train 0.54, test 0.33
['VISp']: C 21.54434690031882, train 1.00, test 0.51
['VISpm']: C 0.005994842503189409, train 0.86, test 0.38
['VISrl']: C 2.782559402207126, train 1.00, test 0.55


## ... Left stimulus stronger than Right

In [15]:
model_list = []
for region in [["VISa"], ["VISam"], ["VISl"], ["VISp"], ["VISpm"], ["VISrl"]]:
    samples, y = get_samples(region, contrast_list='left_stimulus')
    samples_train, samples_test, y_train, y_test = train_test_split(samples, y, test_size=0.2, random_state=42)
    clf = LogisticRegressionCV(penalty='l2', Cs=10, cv=3, solver='sag', tol=0.0001, max_iter=5000, random_state=42).fit(samples_train, y_train)
    print(f'{region}: C {clf.C_[0]}, train {clf.score(samples_train,y_train):.2f}, test {clf.score(samples_test,y_test):.2f}')
    model_list.append(clf)

['VISa']: C 0.0001, train 0.56, test 0.25
['VISam']: C 0.000774263682681127, train 0.75, test 0.47
['VISl']: C 0.005994842503189409, train 1.00, test 0.67
['VISp']: C 0.000774263682681127, train 0.67, test 0.36
['VISpm']: C 0.3593813663804626, train 1.00, test 0.78
['VISrl']: C 21.54434690031882, train 1.00, test 0.20
